# ACID Properties and Transactions in SQLite

![](../../imgs/acid.png)


## Objectives

This class demonstrates the ACID properties of database systems using SQLite:

- **Atomicity**: All or nothing execution
- **Consistency**: Database must remain valid
- **Isolation**: Transactions should not interfere
- **Durability**: Committed changes must persist

We will simulate common transaction operations and analyze their behavior.

In [1]:
import sqlite3

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create a simple accounts table
cursor.execute('''
    CREATE TABLE accounts (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        balance INTEGER NOT NULL CHECK(balance >= 0)
    )
''')

# Insert initial balances
cursor.executemany('''
    INSERT INTO accounts (name, balance) VALUES (?, ?)
''', [
    ('Alice', 100),
    ('Bob', 100)
])

conn.commit()
print("Database initialized.")


Database initialized.


**Explanation:**

- `CREATE TABLE` defines a new table with constraints.
- `CHECK(balance >= 0)` enforces consistency.
- `executemany` inserts multiple records.
- `commit()` saves the transaction permanently.


## Atomicity

A transaction should either fully happen or not happen at all.
We simulate a transfer that fails midway to test rollback behavior.


In [2]:
def transfer_with_failure(conn):
    try:
        cursor = conn.cursor()
        cursor.execute("BEGIN")  # Start transaction
        cursor.execute("UPDATE accounts SET balance = balance - 50 WHERE name = 'Alice'")
        raise Exception("System crash during transaction")  # Simulated failure
        cursor.execute("UPDATE accounts SET balance = balance + 50 WHERE name = 'Bob'")
        conn.commit()
    except Exception as e:
        print("Error:", e)
        conn.rollback()  # Ensure rollback on error

transfer_with_failure(conn)

cursor.execute("SELECT * FROM accounts")
cursor.fetchall()


Error: System crash during transaction


[(1, 'Alice', 100), (2, 'Bob', 100)]

**Explanation:**

- `BEGIN` starts a transaction.
- If an error occurs mid-transaction, `rollback()` undoes the changes.
- This ensures **atomicity**: either all operations succeed or none do.


## Consistency

Database constraints (like non-negative balances) ensure data remains valid after any transaction.


In [3]:
def violate_consistency(conn):
    try:
        cursor = conn.cursor()
        cursor.execute("BEGIN")
        cursor.execute("UPDATE accounts SET balance = balance - 200 WHERE name = 'Alice'")
        cursor.execute("UPDATE accounts SET balance = balance + 200 WHERE name = 'Bob'")
        conn.commit()
    except Exception as e:
        print("Error:", e)
        conn.rollback()

violate_consistency(conn)

cursor.execute("SELECT * FROM accounts")
cursor.fetchall()


Error: CHECK constraint failed: balance >= 0


[(1, 'Alice', 100), (2, 'Bob', 100)]

**Explanation:**

- This example attempts to make Alice's balance negative.
- The `CHECK(balance >= 0)` constraint fails.
- SQLite rolls back the transaction to maintain **consistency**.


## Isolation

SQLite ensures isolation by default. Two connections working in parallel shouldn't interfere.


In [4]:
conn1 = sqlite3.connect("acid_demo.db", isolation_level="DEFERRED")
conn2 = sqlite3.connect("acid_demo.db", isolation_level="DEFERRED")

c1 = conn1.cursor()
c2 = conn2.cursor()

c1.execute("BEGIN")
c1.execute("UPDATE accounts SET balance = balance - 30 WHERE name = 'Alice'")

c2.execute("BEGIN")
try:
    c2.execute("UPDATE accounts SET balance = balance + 30 WHERE name = 'Bob'")
    conn2.commit()
except Exception as e:
    print("Isolation error:", e)

conn1.commit()

cursor = conn.cursor()
cursor.execute("SELECT * FROM accounts")
cursor.fetchall()


OperationalError: no such table: accounts

**Explanation:**

- Each connection starts its own transaction.
- Changes are not visible to the other transaction until `commit()` is called.
- This shows **isolation** in action.


## Durability

Committed transactions should persist even if the database is closed or the system crashes.


In [5]:
cursor.execute("BEGIN")
cursor.execute("UPDATE accounts SET balance = balance + 10 WHERE name = 'Alice'")
conn.commit()

# Simulate closing and reopening the database
conn.close()
conn = sqlite3.connect("acid_demo.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM accounts")
cursor.fetchall()


[(1, 'Alice', 50), (2, 'Bob', 100)]

**Explanation:**

- Once committed, changes are permanently written.
- Reopening the DB shows Alice's balance updated, confirming **durability**.


## Summary

| ACID Property | Demonstration Summary | SQLite Feature Used |
|---------------|------------------------|---------------------|
| Atomicity     | Rollback on failure    | `BEGIN`, `rollback` |
| Consistency   | Valid data only        | `CHECK` constraints |
| Isolation     | Separate transactions  | Multiple connections |
| Durability    | Changes persist        | `commit()` + reopen |
